In [23]:
!pip install langchain openai

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
cd '/content/drive/MyDrive/workspace/train-llm'

/content/drive/MyDrive/workspace/train-llm


In [29]:
import os
from configparser import ConfigParser

config = ConfigParser()
config.read('./conf.ini')
openai_token=config['secret']['openai_token']
os.environ['OPENAI_API_KEY'] = openai_token

In [35]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(openai_api_key=openai_token)

In [38]:
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [37]:
print(llm.predict("Hi! I'm Aiden"))



Nice to meet you Aiden. How are you today?


In [33]:
text = "What would be a good company name for a company that makes colorful socks?"

print(llm.predict(text))



BrightStride Socks.


# Prompt Template
- 대부분의 LLM app은 사용자 입력을 LLM에 직접 전달하지 않음
- Prompt Template을 통해서 더 큰 텍스트를 LLM에 전달

In [39]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="IT Solutions")

'What is a good name for a company that makes IT Solutions?'

In [40]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant that translates {input_language} to {output_language}."),
    HumanMessagePromptTemplate.from_template("{text}"),
])

chat_prompt.format_messages(input_language="English", output_language="Korean", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

# Output Parser
- LLM의 Raw output을 뒤에 사용할 수 있는 형식으로 변환
- 종류
    1. Convert text 정제화된 정보 (JSON 등)으로 변환
    1. ChatMessage를 단순 String으로 변환
    1. 메시지 외에 반환 값을 문자열로 변환

In [41]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

['hi', 'bye']

# LLM Chain
- 위에 Component를 하나의 체인으로 결합
- Input > Prompt Template > Prompt > LLM > Output > Output Parser > Result

In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("korean traditional food")

['Bibimbap', 'Kimchi', 'Bulgogi', 'Japchae', 'Tteokbokki']